In [ ]:
## Datubāzes pieslēgšana skrāpētājam
from scrapy.conf import settings

settings['MONGODB_SERVER'] = "localhost"
settings['MONGODB_PORT'] = 27017
## Lai saglabātu rezultātus, ir nepieciešams izveidot datubāzi mongodb "marketplace_scraper"
settings['MONGODB_DB'] = "marketplace_scraper"

In [ ]:
import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy.utils.project import get_project_settings
from marketplace_scraper.spiders.ss_cars_spider import SsSpider
from marketplace_scraper.spiders.autoplius_cars_spider import AutopliusSpider

process = CrawlerProcess(get_project_settings())

# Definēsim, kādus URL gribam apstaigāt SS.COM
SSurls = [
    'https://www.ss.com/lv/transport/cars/bmw/',
    'https://www.ss.com/lv/transport/cars/audi/'
]
## Šis ir nepieciešams, lai zinātu, kādas vecās kolekcijas ir jāizmet, lai netiktu sabojāta statistika
# TODO: Pārveidot, lai katrai jaunai kolekcijai pieliktu timestamp un tādā veidā tās varētu atšķirt
SSdropCollections = [
    'bmw',
    'audi'
]

# Definēsim, kādus URL gribam apstaigāt AUTOPLIUS.LT
APurls = [
    'https://en.autoplius.lt/ads/used-cars/bmw',
    'https://en.autoplius.lt/ads/used-cars/audi'
]
APdropCollections = [
    'bmw_autoplius',
    'audi_autoplius'
]

SS = SsSpider(scrapy.Spider)
AP = AutopliusSpider(scrapy.Spider)

# Ievietojam augstāk definētos URL un kolekcijas
SS.urls = SSurls
SS.dropCollections = SSdropCollections

AP.urls = APurls
AP.dropCollections = APdropCollections

In [ ]:
## Palaidīsim skrāpētājus
## !!! UZMANĪBU !!! VAR AIZŅEMT ILGU LAIKU !!!

process.crawl(SS)
process.crawl(AP)
process.start()

## Ja nesanāk palaist, tad projektā var palaist failu tieši 'run.py' vai
## projektā palaist komandas "scrapy crawl ss_cars" un "scrapy crawl autoplius"

In [ ]:
from marketplace_scraper.analyzing.analyze_cars import AnalyzeData
from marketplace_scraper.analyzing.analyze_cars import ItemCollection
from marketplace_scraper.analyzing.car_brands.car_brand import CarBrand

Data = AnalyzeData()

db = Data.db
#Salīdzina vienādos modeļus starp portāliem
collections1 = [
    ItemCollection(CarBrand(db, 'bmw', [
        '320'
    ]), 'SS.COM'),
    ItemCollection(CarBrand(db, 'bmw_autoplius', [
        '320'
    ]), 'Autoplius.lt')
]
Data.collections = collections1
Data.analyzeData()


In [ ]:
#Salīdzina dažādus modeļus vienā portālā
collections2 = [
    ItemCollection(CarBrand(db, 'bmw', [
        '320'
    ]), 'SS.COM'),
    ItemCollection(CarBrand(db, 'bmw', [
        '330'
    ]), 'SS.COM')
]
Data.collections = collections2
Data.analyzeData()


In [ ]:
#Salīdzina vairāk kā 2 modeļus
collections3 = [
    ItemCollection(CarBrand(db, 'bmw', [
        '320'
    ]), 'SS.COM'),
    ItemCollection(CarBrand(db, 'bmw', [
        '318'
    ]), 'SS.COM'),
    ItemCollection(CarBrand(db, 'bmw', [
        '325'
    ]), 'SS.COM'),
    ItemCollection(CarBrand(db, 'bmw', [
        '330'
    ]), 'SS.COM')
]
Data.collections = collections3
Data.analyzeData()